<a href="https://colab.research.google.com/github/hieubkset/Colab-Notebooks/blob/master/save_and_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lưu và Nạp model**

Hãy tưởng tượng bạn đang training một model được $7 \times 7 = 49$ ngày, độ chính xác đã được $96.69$%, bỗng dưng mất điện. Bạn sẽ phải train lại từ đầu?

Câu trả lời là không nếu bạn thực hiện lưu (save) model định kỳ. Khi đó, bạn chỉ cần nạp (load) và tiếp tục quá trình training.

Tổng quát hơn, khi nào cần lưu và nạp model:

+ Để chia sẻ model với mọi người.
+ Để triển khai, ứng dụng các model, ví dụ: đưa model lên web hay xuống các thiết bị nhúng.
+ Để tiếp tục quá trình training. 

Có hai cách để chia sẻ model:

+ Cách 1: Lưu lại các tham số (trained weights/parameters) và chia sẻ code tạo model.
+ Cách 2: Lưu lại toàn bộ các thông tin của model, bao gồm: kiến trúc của model, các tham số, trạng thái của optimizer, v.v.

Cách 2 đảm bảo nếu tiếp tục training có thể bắt đầu tại đúng điểm đã kết thúc vì nó lưu lại cả trạng thái của optimizer.

Trong bài này, chúng ta sẽ tìm hiểu cách lưu (save) và nạp (load) model sử dụng các API của tf.keras.

## **Chương trình**

**Khai báo các thư viện**

In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

!pip install pyyaml h5py

In [0]:
import os
import tensorflow as tf
from tensorflow import keras

print("Version: ", tf.__version__)

**Chuẩn bị data**

Để giảm thời gian training và testing, chúng ta chỉ sử dụng 1000 example trong cả tập train và tập test.

In [0]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000] / 255.0
test_images = test_images[:1000] / 255.0

**Khởi tạo model**

Vì chúng ta sẽ tạo nhiều model có cùng cầu hình nên ta sẽ viết code để có thể tái sử dụng:

In [4]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10))

    model.compile(optimizer='Adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### **1. Lưu và Nạp các tham số của model bằng callback**

#### **1.1. Lưu tham số cuối cùng** 

##### **a. Lưu các tham số**

Sử dụng `keras.callbacks.ModelCheckpoint` để lưu các tham số (weights):

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

In [6]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


`cp_callback` sẽ được gọi tại cuối mỗi epoch, nó sẽ lưu các tham số của model thành nhiều file (`cp.ckpt.data-00001-of-00002`, `cp.ckpt.data-00000-of-00002`,  `cp.ckpt.index`) vào thư mục `checkpoint_dir`. Quá trình này thực hiện ghi đè nên chúng ta chỉ lưu lại các tham số của model ở epoch cuối cùng khi chương trình kết thúc.  

##### **b. Nạp tham số của model**

Chúng ta tạo model mới và đánh giá accuracy:

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.3174 - accuracy: 0.0970
Untrained model, accuracy:  9.70%


Model được tạo mới chỉ có accuracy xấp xỉ 10%.

Thực hiện nạp các tham số đã lưu vào model mới này và đánh giá accuracy:

In [8]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4066 - accuracy: 0.8720
Restored model, accuracy: 87.20%


Model mới sau khi đã nạp các tham số cho accuracy khoảng 90%.

#### **1.2. Lưu tham số tại mỗi epoch**

##### **a. Lưu các tham số**

Ở đoạn code dưới đây, các tham số sẽ được lưu sau mỗi 5 epoch.

In [0]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              period=5,
                                              verbose=1)

model = create_model()

# thực hiện lưu thủ công tham số của model khi mới tạo
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)

In [10]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


##### **b. Nạp tham số tại một epoch**

In [11]:
model = create_model()

model.load_weights(checkpoint_path.format(epoch=5))

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4821 - accuracy: 0.8480
Restored model, accuracy: 84.80%


##### **c. Nạp tham số tại epoch cuối cùng**

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [13]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4711 - accuracy: 0.8790
Restored model, accuracy: 87.90%


### **2. Lưu và Nạp tham số model thủ công**

Để lưu và nạp tham số thủ công (không sử dụng callback), ta sử dụng hàm `save_weights` và `load_weights`:

In [14]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4711 - accuracy: 0.8790
Restored model, accuracy: 87.90%


In [15]:
!ls checkpoints

checkpoint  my_checkpoint.data-00000-of-00001  my_checkpoint.index


### **3. Lưu và Nạp toàn bộ model**

Có hai định dạng để lưu toàn bộ model là `SavedModel` (mặc định của Tensorflow) và `HDF5`.

#### **3.1. Lưu và Nạp toàn bộ model định dạng `SavedModel`**:

In [0]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

In [17]:
!ls saved_model

!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [18]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4435 - accuracy: 0.8640
Restored model, accuracy: 86.40%


#### **3.2. Lưu và Nạp toàn bộ model định dạng `HDF5`**

In [0]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

In [21]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4224 - accuracy: 0.8630
Restored model, accuracy: 86.30%
